In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 8
fig_height = 4
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/home/diego/local_repos/AGEC652_2024/slides/lecture_6_3"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| include: false
using Pkg
Pkg.activate(".")
Pkg.instantiate()
# Pkg.add("JuMP")
# Pkg.add("Ipopt")
# Pkg.add("Optim")
# Pkg.add("Statistics")
# Pkg.add("Plots")
# Pkg.add("ForwardDiff")
# Pkg.add("LinearAlgebra")
# Pkg.add("LaTeXStrings")
# Pkg.add("GLM")
# Pkg.add("DataFrames")
# Pkg.add("Distributions")

  Activating project at `~/local_repos/AGEC652_2024/slides/lecture_6_3`


In [3]:
using CSV, DataFrames
df = CSV.read("transportation_mode_survey.csv", DataFrame);

In [4]:
println(df[1:8, :])

8×10 DataFrame
 Row │ city     price_bus  price_train  stops_bus  stops_train  density  hhsize  homeown  d_ib   d_it  
     │ String7  Float64    Float64      Int64      Int64        Float64  Int64   Int64    Int64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ nr_1          6.49          0.0         36            0     5.3        2        0      1      0
   2 │ nr_2          8.44          0.0         49            0     7.7        2        1      0      0
   3 │ nr_3          5.54          0.0         32            0     4.35       3        1      0      0
   4 │ nr_4          7.99          0.0         44            0     6.29       4        0      0      0
   5 │ nr_5          4.97          0.0         27            0     4.07       2        1      0      0
   6 │ nr_6          3.23          0.0         18            0     2.43       3        1      0      0
   7 │ nr_7          3.72          0.0         17      

In [5]:
df_r = filter(row -> startswith(row[:city], "r_"), df);

In [6]:
N_r = nrow(df_r);
ones_column = ones(N_r);

In [7]:
X_b = df_r[:, [:price_bus, :stops_bus, :density, :hhsize, :homeown]]
X_b = Matrix(X_b) # This converts the DataFrame into a matrix
X_b = hcat(ones_column, X_b); # This concatenates the vector of 1s and X_b horizontally

In [8]:
X_b[1:5, :]

5×6 Matrix{Float64}:
 1.0  6.49  36.0  5.3   3.0  1.0
 1.0  8.44  49.0  7.7   2.0  1.0
 1.0  5.54  32.0  4.35  3.0  1.0
 1.0  7.99  44.0  6.29  3.0  0.0
 1.0  4.97  27.0  4.07  3.0  0.0

In [9]:
X_t = df_r[:, [:price_train, :stops_train, :density, :hhsize, :homeown]]
X_t = Matrix(X_t)
X_t = hcat(ones_column, X_t)

60000×6 Matrix{Float64}:
 1.0  12.52  25.0  5.3   3.0  1.0
 1.0   6.11  16.0  7.7   2.0  1.0
 1.0  13.35  30.0  4.35  3.0  1.0
 1.0  10.09  24.0  6.29  3.0  0.0
 1.0   6.01  12.0  4.07  3.0  0.0
 1.0   6.68  16.0  2.43  2.0  0.0
 1.0  11.49  24.0  1.87  2.0  1.0
 1.0   7.76  16.0  7.32  2.0  1.0
 1.0   8.52  19.0  2.34  1.0  1.0
 1.0   6.35  14.0  5.53  3.0  1.0
 1.0  13.25  27.0  2.07  4.0  0.0
 1.0  14.23  29.0  7.39  1.0  1.0
 1.0  13.27  27.0  2.84  4.0  1.0
 ⋮                            ⋮
 1.0  11.3   25.0  1.15  2.0  1.0
 1.0  12.42  24.0  1.47  2.0  1.0
 1.0   8.06  16.0  6.15  2.0  0.0
 1.0  11.88  27.0  1.45  5.0  0.0
 1.0  12.09  26.0  3.93  3.0  0.0
 1.0   8.24  18.0  1.95  3.0  0.0
 1.0  12.4   26.0  7.14  4.0  1.0
 1.0  11.11  23.0  3.0   3.0  1.0
 1.0  13.79  28.0  4.38  2.0  0.0
 1.0  12.55  28.0  1.72  4.0  1.0
 1.0   9.27  17.0  5.14  2.0  1.0
 1.0   7.88  16.0  2.26  3.0  0.0

In [10]:
d_b = Matrix(df_r[:, [:d_ib]]);
d_t = Matrix(df_r[:, [:d_it]]);

In [11]:
function neg_log_likelihood(β; X_b = X_b, X_t = X_t, d_b = d_b, d_t = d_t)
    # Unpack separate vector of parameters
    β_b = β[1:6]
    β_t = β[7:12]
    # Calculate V_ik = X_ik * β_k
    V_ib = X_b * β_b # Note that his is a matrix multiplication
    V_it = X_t * β_t # And this one, too
    # Calculate the denominator of the probabilities (the 2nd term in l)
    log_denom = log.(1.0 .+ exp.(V_ib) .+ exp.(V_it))
    # Calculate the whole l
    neg_l = -sum(d_b .* V_ib .+ d_t .* V_it .- log_denom)
    return neg_l
end 

neg_log_likelihood (generic function with 1 method)

In [12]:
true_β_b = [-1.0, -0.8,  0.1,  0.3, -0.2, -0.3];
true_β_t = [-0.6, -0.8,  0.2,  0.6, -0.1, -0.2];
true_β = vcat(true_β_b, true_β_t);
-neg_log_likelihood(true_β)

-32662.007134423682

In [13]:
-neg_log_likelihood(zeros(12))

-65916.73732008654

In [14]:
using Optim
# Let's give it a vector of zeros for the initial guess
res = optimize(neg_log_likelihood, zeros(12), BFGS())

 * Status: success

 * Candidate solution
    Final objective value:     3.212633e+04

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 9.99e-10 ≰ 0.0e+00
    |x - x'|/|x'|          = 7.97e-10 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 9.91e-06 ≰ 1.0e-08

 * Work counters
    Seconds run:   5  (vs limit Inf)
    Iterations:    24
    f(x) calls:    115
    ∇f(x) calls:   115


In [15]:
res.minimum

32126.33295639354

In [16]:
res_1 = optimize(neg_log_likelihood, ones(12), BFGS());
res_1.minimum

32126.33295639354

In [17]:
res_2 = optimize(neg_log_likelihood, 2*ones(12), BFGS());
res_2.minimum

32126.332956393537

In [18]:
β_MLE = res.minimizer;
β_MLE'

1×12 adjoint(::Vector{Float64}) with eltype Float64:
 -1.25302  -0.981234  0.109963  0.486675  …  0.744581  -0.13079  -0.215491

In [19]:
β_MLE[2]

-0.9812342431109393

In [20]:
β_MLE[8]

-0.9591216015843176

In [21]:
function restr_neg_log_likelihood(β; X_b = X_b, X_t = X_t, d_b = d_b, d_t = d_t)
    # We just take element 2 in β and insert it again between elements 7 and 8
    β_b = β[1:6] # The usual
    β_t = vcat(β[7], β[2], β[8:11]) # Repeat the price coefficient
    # Stack it again
    β_restricted = vcat(β_b, β_t) 
    # The rest is like the previous one, so we can just call our other function!
    neg_l = neg_log_likelihood(β_restricted; X_b = X_b, X_t = X_t, d_b = d_b, d_t = d_t)
    return neg_l
end 

restr_neg_log_likelihood (generic function with 1 method)

In [22]:
res_restr = optimize(restr_neg_log_likelihood, zeros(11), BFGS())

 * Status: success

 * Candidate solution
    Final objective value:     3.212641e+04

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 1.20e-06 ≰ 1.0e-08

 * Work counters
    Seconds run:   6  (vs limit Inf)
    Iterations:    35
    f(x) calls:    136
    ∇f(x) calls:   136


In [23]:
β_MLE_restr = res_restr.minimizer;
β_MLE_restr'

1×11 adjoint(::Vector{Float64}) with eltype Float64:
 -1.26834  -0.962045  0.10644  0.49067  …  0.744867  -0.130826  -0.215556

In [24]:
l_U = -res.minimum;
l_R = -res_restr.minimum;

In [25]:
LR_stat = 2 * (l_U - l_R)

0.15121039505902445

In [26]:
using Distributions
crit_value = quantile(Chisq(1), 0.95);
println("Critical value: $crit_value . Test statistic: $LR_stat")

Critical value: 3.8414588206941245 . Test statistic: 0.15121039505902445


In [27]:
β_b_hat = β_MLE_restr[1:6]; # The usual
β_t_hat = vcat(β_MLE_restr[7], β_MLE_restr[2], β_MLE_restr[8:11]); # Repeat the price coefficient
β_hat = vcat(β_b_hat, β_t_hat);

In [28]:
true_β'

1×12 adjoint(::Vector{Float64}) with eltype Float64:
 -1.0  -0.8  0.1  0.3  -0.2  -0.3  -0.6  -0.8  0.2  0.6  -0.1  -0.2

In [29]:
β_hat'

1×12 adjoint(::Vector{Float64}) with eltype Float64:
 -1.26834  -0.962045  0.10644  0.49067  …  0.744867  -0.130826  -0.215556

In [30]:
true_β'./0.8

1×12 Matrix{Float64}:
 -1.25  -1.0  0.125  0.375  -0.25  -0.375  …  -1.0  0.25  0.75  -0.125  -0.25

In [31]:
β_hat'

1×12 adjoint(::Vector{Float64}) with eltype Float64:
 -1.26834  -0.962045  0.10644  0.49067  …  0.744867  -0.130826  -0.215556

In [32]:
using ForwardDiff, LinearAlgebra
Im = -ForwardDiff.hessian(restr_neg_log_likelihood, β_MLE_restr) # Flip the signs
V = inv(Im);
SEs = sqrt.(diag(-V));

In [33]:
res_df = DataFrame(
  Coefficient = ["β_b0", "β_b1=β_t1", "β_b2", "β_b3", "β_b4", "β_b5", "β_t0", "β_t2", "β_t3", "β_t4", "β_t5"],
  Estimate = β_MLE_restr,
  StdError = SEs,
  CI_lower = β_MLE_restr .+ quantile(Normal(), 0.025) .* SEs,
  CI_upper = β_MLE_restr .+ quantile(Normal(), 0.975) .* SEs
)
println(res_df)

11×5 DataFrame
 Row │ Coefficient  Estimate   StdError    CI_lower    CI_upper  
     │ String       Float64    Float64     Float64     Float64   
─────┼───────────────────────────────────────────────────────────
   1 │ β_b0         -1.26834   0.0649339   -1.3956     -1.14107
   2 │ β_b1=β_t1    -0.962045  0.01838     -0.998069   -0.926021
   3 │ β_b2          0.10644   0.00857776   0.0896282   0.123252
   4 │ β_b3          0.49067   0.0473282    0.397908    0.583432
   5 │ β_b4         -0.256119  0.012644    -0.280901   -0.231337
   6 │ β_b5         -0.385548  0.0284063   -0.441223   -0.329873
   7 │ β_t0         -0.741272  0.0818007   -0.901598   -0.580946
   8 │ β_t2          0.236894  0.00857381   0.22009     0.253698
   9 │ β_t3          0.744867  0.0096659    0.725923    0.763812
  10 │ β_t4         -0.130826  0.0125632   -0.15545    -0.106203
  11 │ β_t5         -0.215556  0.0301444   -0.274638   -0.156474


In [34]:
# Predictions
V_b_hat = X_b * β_hat[1:6];
V_t_hat = X_t * β_hat[7:12];
# The denominator is the same for all options
denom_r = 1.0 .+ exp.(V_b_hat) .+ exp.(V_t_hat);

In [35]:
prob_b_r = exp.(V_b_hat) ./ denom_r;
prob_t_r = exp.(V_t_hat) ./ denom_r;
prob_c_r = 1.0 .- prob_b_r .- prob_t_r;

In [36]:
using Statistics
mean_prob_b_r = mean(prob_b_r)

0.09985000000032383

In [37]:
sum(d_b)/N_r

0.09985

In [38]:
mean_prob_t_r = mean(prob_t_r)

0.13886666666597253

In [39]:
sum(d_t)/N_r

0.13886666666666667

In [40]:
# Filter for nr cities
df_nr = filter(row -> startswith(row[:city], "nr_"), df);
N_nr = nrow(df_nr);

In [41]:
# Here we are combining a few steps at once
X_b_nr = Matrix(df_nr[:, [:price_bus, :stops_bus, :density, :hhsize, :homeown]]);
X_b_nr = hcat(ones(nrow(df_nr)), X_b_nr);

In [42]:
X_t_nr = df_nr[:, [:price_bus, :stops_train, :density, :hhsize, :homeown]];
X_t_nr.price_bus = 1.1 .* X_t_nr.price_bus; # 5% more expensive
X_t_nr.stops_train = 10 .* ones(N_nr); # 10 stops
X_t_nr = Matrix(X_t_nr);
X_t_nr = hcat(ones(N_nr), X_t_nr);

In [43]:
# Predictions
V_b_hat_nr = X_b_nr * β_hat[1:6];
V_t_hat_nr = X_t_nr * β_hat[7:12];
# The denominator is the same for all options
denom_nr = 1.0 .+ exp.(V_b_hat_nr) .+ exp.(V_t_hat_nr);

In [44]:
prob_b_nr = exp.(V_b_hat_nr) ./ denom_nr;
prob_t_nr = exp.(V_t_hat_nr) ./ denom_nr;
prob_c_nr = 1.0 .- prob_b_nr .- prob_t_nr;

In [45]:
mean_prob_b_nr = mean(prob_b_nr)

0.09041621616633082

In [46]:
mean_prob_t_nr = mean(prob_t_nr)

0.14923985762722458

In [47]:
mean_prob_c_nr = mean(prob_c_nr)

0.7603439262064445